In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
import numpy as np
import os
import random
from openpyxl import Workbook

In [2]:
number_of_clusters = [2, 3, 4, 5, 6, 7]

In [3]:
pts = ["01", "04", "05", "06", "07", "08", "09", "10", "15", "16", "18", "21", "22"]

In [4]:
def SPEC(y_test, y_pred):
    classes = np.unique(y_test)
    specs = []
    
    for i in classes:
        fp = 0
        tn = 0
        
        for test, pred in zip(y_test, y_pred):
            if pred == i and test != i:
                fp += 1
            if test != i and pred != i:
                tn += 1
        
        try:            
            spec = tn / (tn + fp)
        except:
            spec = 1
            
        specs.append(spec)    
    
    return round(np.mean(specs)*100, 2)

In [5]:
def FPR(y_test, y_pred):
    classes = np.unique(y_test)
    fprs = []
    
    for i in classes:
        fp = 0
        tn = 0
        
        for test, pred in zip(y_test, y_pred):
            if pred == i and test != i:
                fp += 1
            if test != i and pred != i:
                tn += 1
                
        try:            
            fpr = fp / (fp + tn)
        except:
            fpr = 0
            
        fprs.append(fpr)
            
    
    return round(np.mean(fprs), 3)

In [6]:
def recall(y_test, y_pred):
    classes = np.unique(y_test)
    recalls = []
    
    for i in classes:
        fn = 0
        tp = 0
        
        for test, pred in zip(y_test, y_pred):
            if pred == i and test == i:
                tp += 1
            if pred != i and test == i:
                fn += 1
                
        try:            
            recall = tp / (tp + fn)
        except:
            recall = 1
            
        recalls.append(recall)
    
    return round(np.mean(recalls)*100, 2)

## Only patient

In [7]:
length = 60

features_path = f"/home/npe/seizure-prediction/Data/Interval-300-classes-length-{length}/Features/classification-1-5"

wb = Workbook()
ws = wb.active

ws.append(['Patient', 'Accuracy(%)', 'Precision(%)', 'Recall(%)', 'Specificity(%)', 'FPR(/h)'])

for cluster in number_of_clusters:
    all_accs, all_precs, all_sens, all_specs, all_fprs = [], [], [], [], []
    
    cluster_path = f"{features_path}/{cluster}-clusters"
    classes = os.listdir(cluster_path)
    print(f"Cluster {cluster}:")
    
    for pt in pts:
        x, y, x_train, x_test, y_train, y_test = [], [], [], [], [], []
        
        for _class in classes:
            if _class[0] == 'c':
#                 print(_class)
                class_path = f"{cluster_path}/{_class}"
                class_data = os.listdir(class_path)

                for data in class_data:
                    if data[2:4] == pt:
                        try:
                            x.append(np.load(f"{class_path}/{data}"))
                            y.append(data.split('-')[1][5:])
                        except:
                            print('*****')
                        
        x, y = shuffle(x, y)             
                    
        # Test split
        kf = KFold(n_splits=10)
        
        accs, precs, sens, specs, fprs = [], [], [], [], []
        
        for train_index, test_index in kf.split(x):
#             print("TRAIN:", len(train_index), "TEST:", len(test_index))
            x_train, x_test, y_train, y_test = [], [], [], []
            
            for i in range(len(x)):
                if i in test_index:
                    x_test.append(x[i])
                    y_test.append(y[i])

                else:
                    x_train.append(x[i])
                    y_train.append(y[i])

            x_train, x_test = np.array(x_train), np.array(x_test)
            y_train, y_test = np.array(y_train), np.array(y_test)

#             print(x_train.shape)

            reshape_to = x_train.shape[1] * x_train.shape[2]
            x_train = np.array([i.reshape((reshape_to)) for i in x_train])

            reshape_to = x_test.shape[1] * x_test.shape[2]
            x_test = np.array([i.reshape((reshape_to)) for i in x_test])

#             print(x_train.shape, y_train.shape)

            rfc = RandomForestClassifier(max_depth=2, random_state=0)
            rfc.fit(x_train, y_train)
            
            # Accuracy
            acc = rfc.score(x_test, y_test)
            accs.append(round(acc*100, 2))
            
            y_pred = rfc.predict(x_test)
            
            # Precision
            prec = precision_score(y_test, y_pred, average='macro')
            precs.append(round(prec*100, 2))
            
            # Recall (SEN)
            rec = recall_score(y_test, y_pred, average='macro')
            sens.append(round(rec*100, 2))
            
            # SPEC
            spec = SPEC(y_test, y_pred)
            specs.append(spec)
            
            # FPR
            fpr = FPR(y_test, y_pred)
            fprs.append(fpr)

        # Calculating the averages
        pt_acc = round(np.mean(accs), 2)
        all_accs.append(pt_acc)
        
        pt_prec = round(np.mean(precs), 2)
        all_precs.append(pt_prec)
        
        pt_rec = round(np.mean(sens), 2)
        all_sens.append(pt_rec)
        
        pt_spec = round(np.mean(specs), 2)
        all_specs.append(pt_spec)
        
        pt_fpr = round(np.mean(fprs), 3)
        all_fprs.append(pt_fpr)
        
        
        print(f"Pt-{pt}:")
        print(f"accuracy: {pt_acc}\nprecision: {pt_prec}\nrecall: {pt_rec}\nSPEC: {pt_spec}\nFPR: {pt_fpr}\n")
        
        row = [f"Pt-{pt}", pt_acc, pt_prec, pt_rec, pt_spec, pt_fpr]
            
        ws.append(row)
        
    ws.append(['Average', round(np.mean(all_accs), 2), round(np.mean(all_precs), 2), round(np.mean(all_sens), 2), round(np.mean(all_specs), 2), round(np.mean(all_fprs), 3)])
    ws.append(['*'])
    
wb.save(f"Results/{length}/classes_1_2_3_4_5_6_7_8_9_10.xlsx")

Cluster 2:
1 288 2590
5 288 2590
5 288 2590
1 288 2590
5 288 2590
5 288 2590
1 288 2590
5 288 2590
1 287 2591
1 287 2591
Pt-01:
accuracy: 81.72
precision: 81.71
recall: 81.78
SPEC: 81.78
FPR: 0.182

1 288 2590
1 288 2590
1 288 2590
1 288 2590
5 288 2590
1 288 2590
1 288 2590
5 288 2590
5 287 2591
5 287 2591
Pt-04:
accuracy: 78.28
precision: 78.52
recall: 78.25
SPEC: 78.25
FPR: 0.218

5 288 2590
5 288 2590
5 288 2590
5 288 2590
5 288 2590
5 288 2590
5 288 2590
1 288 2590
5 287 2591
1 287 2591
Pt-05:
accuracy: 62.89
precision: 63.23
recall: 62.86
SPEC: 62.86
FPR: 0.372

*****
1 576 5179
1 576 5179
1 576 5179
1 576 5179
1 576 5179
1 575 5180
1 575 5180
5 575 5180


KeyboardInterrupt: 